<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Cleaning_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import every dataset

In [1]:
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

import pandas as pd
from PIL import Image
from torchvision import transforms
import numpy as np

Saving train_df_images.json to train_df_images.json


Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv


In [2]:
train_df = pd.read_csv('[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep='\t')

In [3]:
train_df_images = pd.read_json('train_df_images.json', orient="records", lines=True)
train_df_images

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],..."
...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,None
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,None
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,"[[[61, 117, 223], [64, 118, 226], [65, 119, 22..."
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,"[[[153, 147, 44], [209, 198, 54], [201, 192, 5..."


# Merge 2 datasets

In [4]:
train_df.columns.to_list()

['item', 'name', 'description', 'type', 'category', 'subcategory', 'label']

In [5]:
# Get the dataset we first had
initial_df = train_df
initial_variables = train_df.columns.to_list()

# Get the datasets with the features we created
df_to_merge = [train_df_images]

# Merge all of them, in a single dataframe
for df in df_to_merge:
  initial_df = pd.merge(left= initial_df, right= df, on=initial_variables, how='inner')

# Visualisation :
initial_df

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,"[[[106, 88, 76], [114, 96, 84], [115, 96, 84],..."
...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,None
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,None
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,"[[[61, 117, 223], [64, 118, 226], [65, 119, 22..."
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,"[[[153, 147, 44], [209, 198, 54], [201, 192, 5..."


# Cleaning

## Labels

In [6]:
initial_df['label'].describe()

,label
count,6244
unique,9
top,cultural exclusive
freq,2685


In [7]:
initial_df['label'].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic', 'cultural', 'cult', nan, 'cultural agn',
       'cultural represent', 'cultural ex', 'cultural ag'], dtype=object)

In [8]:
initial_df[['item', 'label']].groupby("label").count()

,item
label,
cult,1
cultural,5
cultural ag,1
cultural agn,3
cultural agnostic,1862
cultural ex,1
cultural exclusive,2685
cultural represent,1
cultural representative,1685


We are suppose to have only 3 different labels

- Cultural Exclusive (C.E.)
- Cultural Agnostic (C.A.)
- Cultural Representative (C.R.)

Instead, we have seen that there are 9 different labels in the dataset, plus some NaN. So we have to clean the data, in order to get only 3 differents labels. Here is the solution we propose :

First step :

We distinguish the 3 labels we want (C.A., C.E., C.R.) and also labels that are close to this ('Cultural agn', 'cultural represent', 'cultural ex', 'cultural ag'). So we propose the following mapping :

- 'cultural agn' : C.A.
- 'cultural represent' : C.R.
- 'cultural ex' : C.E.
- 'cultural ag' : C.A.

Second step :

For the other labels ('cultural', 'cult', nan), we propose to apply the rule that was define to build the dataset (see https://huggingface.co/datasets/sapienzanlp/nlp2025_hw1_cultural_dataset) : Ask ChatGPT-o3 !

First step :

In [9]:
to_correct = initial_df[initial_df["label"].isin(['cultural agn','cultural represent', 'cultural ex', 'cultural ag'])][['item', 'label']]
to_correct

,item,label
1545,http://www.wikidata.org/entity/Q86135347,cultural agn
3698,http://www.wikidata.org/entity/Q509900,cultural represent
4391,http://www.wikidata.org/entity/Q643677,cultural ex
4819,http://www.wikidata.org/entity/Q1711593,cultural agn
5731,http://www.wikidata.org/entity/Q25618,cultural ag
5968,http://www.wikidata.org/entity/Q30405,cultural agn


In [10]:
mapping_correction = {
  'cultural agn' : 'cultural agnostic',
  'cultural represent' : 'cultural representative',
  'cultural ex' : 'cultural exclusive',
  'cultural ag' : 'cultural agnostic'
}

In [11]:
to_correct['label'] = to_correct['label'].map(mapping_correction)

In [12]:
# We update the transformations to the initial dataset
initial_df.set_index('item', inplace=True)
to_correct.set_index('item', inplace=True)

initial_df.update(to_correct)
initial_df.reset_index(inplace=True)


# We check if the updates are corrects
initial_df[1543:1547]

,item,name,description,type,category,subcategory,label,image
1543,http://www.wikidata.org/entity/Q27503001,professional athlete,person who earns their living from sports,concept,sports,athlete,cultural agnostic,None
1544,http://www.wikidata.org/entity/Q107690317,competition climber,climber who competes in IFSC and Olympic climb...,concept,sports,athlete,cultural agnostic,"[[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 2], ..."
1545,http://www.wikidata.org/entity/Q86135347,climber,person who practices climbing,concept,sports,athlete,cultural agnostic,None
1546,http://www.wikidata.org/entity/Q28971125,cheerleader,performer who leads crowd support at sports ev...,concept,sports,athlete,cultural representative,None


Second step :

Here are the items that ChatGPT will have to classify :

In [13]:
to_classify = initial_df[(initial_df["label"].isin(["cultural", "cult"])) | (initial_df["label"].isna())][['item', 'name', 'description']]

to_classify

,item,name,description
858,http://www.wikidata.org/entity/Q957033,Sunny Baudelaire,fictional character
1066,http://www.wikidata.org/entity/Q30327019,post and lintel,building system where horizontal elements (bea...
1214,http://www.wikidata.org/entity/Q811361,architectural glass,building material typically used as transparen...
1651,http://www.wikidata.org/entity/Q12014207,trekking,backpacking or hiking
2556,http://www.wikidata.org/entity/Q257907,Ethiopian movement,religious movement in southern Africa
2831,http://www.wikidata.org/entity/Q1136336,Costa Book Awards,annual series of literary awards in five categ...
3236,http://www.wikidata.org/entity/Q67111,Franz Pfemfert,German journalist (1879-1954)
3820,http://www.wikidata.org/entity/Q1089672,The Elm-Chanted Forest,1986 animated film directed by Milan Blažeković
4853,http://www.wikidata.org/entity/Q85755629,Daniel Airlie,novel
5566,http://www.wikidata.org/entity/Q206912,extremophile,organisms specifically adapted to live and sur...


Here is the result for chatgpt :

In [14]:
categories = {
    'http://www.wikidata.org/entity/Q957033': 'cultural representative',
    'http://www.wikidata.org/entity/Q30327019': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q811361': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q12014207': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q257907': 'cultural exclusive',
    'http://www.wikidata.org/entity/Q1136336': 'cultural representative',
    'http://www.wikidata.org/entity/Q67111': 'cultural representative',
    'http://www.wikidata.org/entity/Q1089672': 'cultural representative',
    'http://www.wikidata.org/entity/Q85755629': 'cultural representative',
    'http://www.wikidata.org/entity/Q206912': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q23228': 'cultural agnostic',
    'http://www.wikidata.org/entity/Q3196604': 'cultural representative',
    'http://www.wikidata.org/entity/Q1940624': 'cultural representative'
}

to_classify['label'] = to_classify['item'].map(categories)

In [15]:
# We update the transformations to the initial dataset
initial_df.set_index('item', inplace=True)
to_classify.set_index('item', inplace=True)

initial_df.update(to_classify)
initial_df.reset_index(inplace=True)


# We check if the updates are corrects
initial_df[856:860]

,item,name,description,type,category,subcategory,label,image
856,http://www.wikidata.org/entity/Q929866,Santi Santamaria,Spanish chef (1957–2011),entity,food,cook,cultural exclusive,"[[[3, 3, 5], [3, 3, 5], [3, 3, 5], [3, 3, 5], ..."
857,http://www.wikidata.org/entity/Q899011,Ian Beale,fictional character from the soap opera EastEn...,entity,food,cook,cultural exclusive,"[[[81, 57, 38], [88, 57, 40], [94, 61, 40], [8..."
858,http://www.wikidata.org/entity/Q957033,Sunny Baudelaire,fictional character,entity,food,cook,cultural representative,None
859,http://www.wikidata.org/entity/Q935079,SpongeBob SquarePants,main character from the animated television sh...,entity,food,cook,cultural representative,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


Check if everything is normal now :

In [16]:
initial_df['label'].describe()

,label
count,6251
unique,3
top,cultural exclusive
freq,2687


In [17]:
initial_df['label'].unique()

array(['cultural exclusive', 'cultural representative',
       'cultural agnostic'], dtype=object)

In [18]:
initial_df[['item', 'label']].groupby("label").count()

,item
label,
cultural agnostic,1871
cultural exclusive,2687
cultural representative,1693


## Image